In [2]:
import csv
import pandas as pd

data_path = "./data.csv"
data = list()

with open(data_path) as csv_file:
    reader = csv.reader(csv_file)
    for line, line_data in enumerate(reader):
        i = 0
        row_count = 0
        
        for row, row_data in enumerate(line_data):
            if line == 0:
                if row_data:
                    data.append([row_data])
                    
            if line == 1:
                if not row_data:
                    i += 1
                    row_count = 0
                    continue
                    
                if row_count == 0:
                    data[i].append([row_data])
                    row_count += 1
                    continue
                
                if row_count == 1 or row_count == 2:
                    data[i][1].append(row_data)
                    row_count += 1
                    continue
                
            if line == 2:
                if not row_data:
                    i += 1
                    continue
                    
                data[i][1].append(row_data)
                
            if line == 3:
                if not row_data:
                    i += 1
                    continue
                    
                data[i][1].append(row_data)
                
            if line == 4:
                if not row_data:
                    i += 1
                    continue
                    
                data[i][1].append(row_data)

            if line == 5:
                if row_data:
                    if row_data == '0':
                        data[i].append([1, 0])
                    elif row_data == '1':
                        data[i].append([0, 1])
                    i += 1

In [3]:
import tensorflow.compat.v1 as tf
import numpy as np

# Use tensorflow v1
tf.disable_v2_behavior()
tf.set_random_seed(777)

learning_rate = 0.2

x_data = []
y_data = []

for row in data:
    x_data.append(row[1])
    y_data.append(row[2])

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

# Shape of image 3 * 4 = 12
X = tf.placeholder(tf.float32, [None, 12], name="x")
Y = tf.placeholder(tf.float32, [None, 2], name="y")

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.truncated_normal([12, 20]), name='weight1')
    b1 = tf.Variable(tf.truncated_normal([20]), name='bias1')
    l1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    
    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", l1)
    
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.truncated_normal([20, 20]), name='weight2')
    b2 = tf.Variable(tf.truncated_normal([20]), name='bias2')
    l2 = tf.sigmoid(tf.matmul(l1, W2) + b2)
    
    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Layer2", l2)

with tf.name_scope("Layer3"):
    W3 = tf.Variable(tf.truncated_normal([20, 20]), name="weight3")
    b3 = tf.Variable(tf.truncated_normal([20]), name="bias3")
    l3 = tf.sigmoid(tf.matmul(l2, W3) + b3)
        
    tf.summary.histogram("W3", W3)
    tf.summary.histogram("b3", b3)
    tf.summary.histogram("Layer3", l3)

with tf.name_scope("Layer4"):
    W4 = tf.Variable(tf.truncated_normal([20, 2]), name="weight4")
    b4 = tf.Variable(tf.truncated_normal([2]), name="bias4")
    Y_pred = tf.sigmoid(tf.matmul(l3, W4) + b4)
    
    tf.summary.histogram("W4", W4)
    tf.summary.histogram("b4", b4)
    tf.summary.histogram("Layer4", Y_pred)

# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(Y_pred) + (1 - Y) * tf.log(1 - Y_pred))
    tf.summary.scalar("Cost", cost)

# Loss derivative
d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)

# Layer4
d_sigma4 = Y_pred * (1 - Y_pred)
d_a4 = d_Y_pred * d_sigma4
d_p4 = d_a4
d_b4 = d_a4
d_W4 = tf.matmul(tf.transpose(l3), d_p4)

# Layer4-Mean
d_b4_mean = tf.reduce_mean(d_b4, axis=[0])
d_W4_mean = d_W4 / tf.cast(tf.shape(l3)[0], dtype=tf.float32)

# Layer3
d_l3 = tf.matmul(d_p4, tf.transpose(W4))
d_sigma3 = l3 * (1 - l3)
d_a3 = d_l3 * d_sigma3
d_b3 = d_a3
d_p3 = d_a3
d_W3 = tf.matmul(tf.transpose(l2), d_a3)

# Layer3-Mean
d_W3_mean = d_W3 / tf.cast(tf.shape(l2)[0], dtype=tf.float32)
d_b3_mean = tf.reduce_mean(d_b3, axis=[0])

# Layer2
d_l2 = tf.matmul(d_p3, tf.transpose(W3))
d_sigma2 = l2 * (1 - l2)
d_a2 = d_l2 * d_sigma2
d_b2 = d_a2
d_p2 = d_a2
d_W2 = tf.matmul(tf.transpose(l1), d_a2)

# Layer2-Mean
d_W2_mean = d_W2 / tf.cast(tf.shape(l1)[0], dtype=tf.float32)
d_b2_mean = tf.reduce_mean(d_b2, axis=[0])

# Layer1
d_l1 = tf.matmul(d_p2, tf.transpose(W2))
d_sigma1 = l1 * (1 - l1)
d_a1 = d_l1 * d_sigma1
d_b1 = d_a1
d_p1 = d_a1
d_W1 = tf.matmul(tf.transpose(X), d_a1)

# Layer1-Mean
d_W1_mean = d_W1 / tf.cast(tf.shape(X)[0], dtype=tf.float32)
d_b1_mean = tf.reduce_mean(d_b1, axis=[0])

# Weight update
step = [
  tf.assign(W4, W4 - learning_rate * d_W4_mean),
  tf.assign(b4, b4 - learning_rate * d_b4_mean),
    
  tf.assign(W3, W3 - learning_rate * d_W3_mean),
  tf.assign(b3, b3 - learning_rate * d_b3_mean),
    
  tf.assign(W2, W2 - learning_rate * d_W2_mean),
  tf.assign(b2, b2 - learning_rate * d_b2_mean),
    
  tf.assign(W1, W1 - learning_rate * d_W1_mean),
  tf.assign(b1, b1 - learning_rate * d_b1_mean)
]

# Accuracy computation
# Get the maximum value, and check the value is equal
predicted = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(predicted, dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/hand_writings_01
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/hand_writings_01")
    writer.add_graph(sess.graph)  # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    print("shape", sess.run(tf.shape(X)[0], feed_dict={X: x_data}))

    for i in range(10001):
        _, summary, cost_val = sess.run([step, merged_summary, cost], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=i)
        
        if i % 1000 == 0:
            print(i, cost_val)

    # Accuracy report
    h, c, a = sess.run([Y_pred, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

Instructions for updating:
non-resource variables are not supported in the long term


2022-06-14 00:08:46.433713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-14 00:08:46.464776: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


shape 64
0 1.2298939
1000 0.0033221412
2000 0.0014237069
3000 0.00089472794
4000 0.0006493789
5000 0.00050843315
6000 0.0004171629
7000 0.00035332964
8000 0.0003062319
9000 0.00027006632
10000 0.00024144765

Hypothesis:  [[9.9984801e-01 1.4586814e-04]
 [9.9982941e-01 1.6226848e-04]
 [9.9983287e-01 1.6417154e-04]
 [9.9983460e-01 1.4510957e-04]
 [9.9980760e-01 1.9956924e-04]
 [9.9974597e-01 2.3616935e-04]
 [9.9932814e-01 5.1284319e-04]
 [9.9949181e-01 4.3780828e-04]
 [9.9934888e-01 5.0286279e-04]
 [9.9962497e-01 3.5692521e-04]
 [9.9981904e-01 1.8410097e-04]
 [9.9979132e-01 1.9619052e-04]
 [9.9969274e-01 3.6553096e-04]
 [9.9957168e-01 4.9370027e-04]
 [9.9955708e-01 4.2856985e-04]
 [9.9985254e-01 1.4457577e-04]
 [9.9981117e-01 1.9087597e-04]
 [9.9980658e-01 1.9926995e-04]
 [9.9985462e-01 1.4321615e-04]
 [9.9978870e-01 1.8506651e-04]
 [9.9980170e-01 1.8207714e-04]
 [9.9983716e-01 1.5512566e-04]
 [9.9977863e-01 2.2013045e-04]
 [9.9970245e-01 2.8124088e-04]
 [9.9984604e-01 1.4634921e-04]
 [9.